In [1]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
train_transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]
)

In [4]:
trainset = datasets.CIFAR100('./data', train = True, transform=train_transform, download = True)
trainloader = DataLoader(trainset,batch_size = 64, shuffle = True, num_workers = 2)

testset = datasets.CIFAR100('./data', train = False, transform = test_transform, download=True)
testloader = DataLoader(testset, batch_size=64, shuffle = False, num_workers = 2)


100%|██████████| 169M/169M [00:20<00:00, 8.16MB/s] 


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class vgg_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2),
            nn.Dropout(0.2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.4)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*4*4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 100)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.fc(x)
        return x


In [6]:
model = vgg_model().to(device)
model

vgg_model(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=20, gamma = 0.5)

In [8]:
epochs = 50

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(trainloader):.4f}")

Epoch 1/50 | Loss: 4.1387
Epoch 2/50 | Loss: 3.6721
Epoch 3/50 | Loss: 3.4267
Epoch 4/50 | Loss: 3.2261
Epoch 5/50 | Loss: 3.0317
Epoch 6/50 | Loss: 2.8747
Epoch 7/50 | Loss: 2.7322
Epoch 8/50 | Loss: 2.6284
Epoch 9/50 | Loss: 2.5275
Epoch 10/50 | Loss: 2.4403
Epoch 11/50 | Loss: 2.3610
Epoch 12/50 | Loss: 2.2988
Epoch 13/50 | Loss: 2.2319
Epoch 14/50 | Loss: 2.1808
Epoch 15/50 | Loss: 2.1248
Epoch 16/50 | Loss: 2.0874
Epoch 17/50 | Loss: 2.0344
Epoch 18/50 | Loss: 1.9991
Epoch 19/50 | Loss: 1.9569
Epoch 20/50 | Loss: 1.9159
Epoch 21/50 | Loss: 1.8512
Epoch 22/50 | Loss: 1.8225
Epoch 23/50 | Loss: 1.7990
Epoch 24/50 | Loss: 1.7927
Epoch 25/50 | Loss: 1.7659
Epoch 26/50 | Loss: 1.7510
Epoch 27/50 | Loss: 1.7350
Epoch 28/50 | Loss: 1.7215
Epoch 29/50 | Loss: 1.7087
Epoch 30/50 | Loss: 1.6882
Epoch 31/50 | Loss: 1.6779
Epoch 32/50 | Loss: 1.6605
Epoch 33/50 | Loss: 1.6387
Epoch 34/50 | Loss: 1.6325
Epoch 35/50 | Loss: 1.6205
Epoch 36/50 | Loss: 1.6043
Epoch 37/50 | Loss: 1.5918
Epoch 38/5

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
acc = 100 * correct/total
print(f"Test accuracy: {acc}%")

Test accuracy: 62.04%


In [10]:
path = './vgg_model100.pth'
torch.save(model.state_dict(), path)